In [1]:
import pandas as pd
import numpy as np
import sys

file_path = 'imdb_network.csv'

# df = pd.DataFrame(new_data, columns=column_names)
# df.set_index('tconst', inplace=True)
# df.to_csv('imdb_years_df.csv', sep=',')

In [2]:
# file headers
# tconst,nconst,primaryTitle,primaryName,primaryProfession
class TitleDictionary:

    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
        self.title_dict = self._create_title_dict()
        self.profession_dict = self._create_profession_dict()

    def _create_title_dict(self):
        title_dict = {}
        data = self.df.values.tolist()

        for entry in data:
            title = entry[2]
            if entry[1] in title_dict:
                title_dict[entry[1]].append(title)
            else:
                title_dict[entry[1]] = [title]

        # print(title_dict['nm6081065']) # debug
        return title_dict 

    def _create_profession_dict(self):
        profession_dict = {}
        data = self.df.values.tolist()

        for entry in data:
            if entry[1] not in profession_dict:
                profession = entry[3]
                if entry[4] == 'actor':
                    profession+='_a'
                else:
                    profession+='_d'

                profession_dict[entry[1]] = profession

        # print(profession_dict['nm6446679']) # debug
        return profession_dict

file_path = 'imdb_network.csv'
test = TitleDictionary(file_path)

In [3]:
td = TitleDictionary('imdb_network.csv')
nconst_title = td.title_dict
nconst_ar_dr = td.profession_dict

print(nconst_title["nm4655646"])

['White as Snow', 'Noose of Ice', 'Point 783', 'Seek and Destroy', 'The Launching']


In [4]:
#Graph Network Creation
'''
adj list
when adding edges
    if less than 2 movies in common, dont add the edge
    if actor->director, reverse the edge to director_actor
    if actor->actor, edge is bidirectional, so add the same edge for the actor

create graph is going to be a dict{dicts{}}, map of maps

add node
'''
class MovieNetwork:
    def __init__(self, name_movie_dict, nconst_ar_dr):
        self.graph = {} 
        self.name_movie_dict = name_movie_dict 
        self.nconst_ar_dr = nconst_ar_dr 
        
    def add_node(self, node):
        self.graph[node] = {}

    def edge_bi(self, node1, node2, weigh):
        if node1 not in self.graph: self.add_node(node1)
        if node2 not in self.graph: self.add_node(node2)

        if node2 not in self.graph[node1]:
            self.graph[node1][node2] = weigh

        if node1 not in self.graph[node2]:
            self.graph[node2][node1] = weigh

    def edge_single(self, node1, node2, weigh):
        if node1 not in self.graph: self.add_node(node1)
        if node2 not in self.graph: self.add_node(node2)
        
        if node2 not in self.graph[node1]:
            self.graph[node1][node2] = weigh

    # nconst_ar_dr, weight=1
    # old parameters, not needed
    def add_edge(self, node1, node2):
        node1_movies = self.name_movie_dict[node1]
        node2_movies = self.name_movie_dict[node2]

        empty_set = set()
        # checking movies in common
        for movie in node1_movies:
            if movie in node2_movies:
                # weight+=1
                empty_set.add(movie)
        
        weight = len(empty_set)
                
        # only proceed if weights are more than 2
        if weight > 2:
            node1_ar_dr = self.nconst_ar_dr[node1]
            node1_profession = node1_ar_dr[len(node1_ar_dr)-1:]

            node2_ar_dr = self.nconst_ar_dr[node2]
            node2_profession = node2_ar_dr[len(node2_ar_dr)-1:]

            if node1_profession == node2_profession:
                self.edge_bi(node1, node2, weight)
            if node1_profession == "a" and node2_profession == "d":
                self.edge_single(node2, node1, weight)
            if node1_profession == "d" and node2_profession == "a":
                self.edge_single(node1, node2, weight)

    def get_graph(self):
        for node1 in self.nconst_ar_dr:
            for node2 in self.nconst_ar_dr:
                if node1 == node2:
                    continue
                if node1 not in self.graph: self.add_node(node1)
                if node2 not in self.graph: self.add_node(node2)
                self.add_edge(node1, node2)
        
        return self.graph

In [5]:
td = TitleDictionary('imdb_network.csv')
nconst_title = td.title_dict
nconst_ar_dr = td.profession_dict

# print(nconst_title['nm1662883'])

movie_network = MovieNetwork(nconst_title, nconst_ar_dr)

graph = movie_network.get_graph()

In [7]:
print(graph['nm4655646'])

{'nm8706223': 5}


In [ ]:
def bfs_queue_helper(queue, nodes):
    for node in nodes: queue.append(node)
    return queue

def bfs(graph, start_node, search_node=None):
    visited = []
    path = []
    queue = [start_node]

    while len(queue) > 0:
        cur_node = queue.pop(0)
        if cur_node not in visited:
            visited.append(cur_node)
            queue = bfs_queue_helper(queue, graph[cur_node])
            path.append(cur_node)

        if  search_node == cur_node:
            return 1  # search node found

    if search_node is not None:
        return 0  # search node not found

    return path 

In [28]:
#old dfs
# dfs_path = []
# dfs_flag = False
# def dfs(graph, start_node, search_node=None):
#     global dfs_path, dfs_flag
#     dfs_path = []
#     dfs_flag = False

#     dfs_recursion(graph, start_node, search_node)

#     if dfs_flag: return 1
#     if search_node is not None:
#         return 0
    
#     return dfs_path

# def dfs_recursion(graph, cur_node, search_node):
#     global dfs_path, dfs_flag

#     if dfs_flag: return # search node found
#     if search_node == cur_node: # search node found, exit out of all dfs
#         dfs_flag = True

#     if cur_node not in dfs_path:
#         dfs_path.append(cur_node)
#     else:
#         return

#     for node in graph[cur_node]:
#         dfs_recursion(graph, node, search_node)

# better codded dfs
discovered = []
dfs_flag = False
# discovered = path
def dfs(graph, start_node, search_node=None):
    global discovered, dfs_flag
    discovered = [start_node]
    dfs_flag = False

    for key in graph[start_node]:
        if key not in discovered:
            dfs_recursion(graph, start_node, search_node)

    if dfs_flag: return 1
    if search_node is not None: return 0
    
    return discovered

def dfs_recursion(graph, cur_node, search_node):
    global discovered, dfs_flag
    if dfs_flag: return         # search node found
    if search_node == cur_node: # search node found, exit out of all dfs
        dfs_flag = True

    discovered.append(cur_node)
    for node in graph[cur_node]:
        if node not in discovered:
            dfs_recursion(graph, node, search_node)

dfs(graph, "nm5822910", search_node="nm5665539")

1

In [24]:
print(graph['nm5822910'])
print(graph['nm1118718'])

{'nm1118718': 5}
{'nm4556923': 3, 'nm9937946': 3, 'nm5665539': 3, 'nm5822910': 5}


In [ ]:
def dijkstra(graph, start_node, end_node):    
    return

In [9]:
# kosaraju - kosaraju - kosaraju - kosaraju - kosaraju - kosaraju - kosaraju - kosaraju
discovered = []
def scc_dfs(graph):
    global discovered
    discovered = []
    for key in graph:
        if key not in discovered:
            scc_dfs_recursion(graph, key)
    return discovered

def scc_dfs_recursion(graph, cur_node):
    global discovered
    discovered.append(cur_node) # discovered
    for node in graph[cur_node]:
        if node not in discovered:
            scc_dfs_recursion(graph, node)

###### second pass################
cur_components = []
def scc_second(graph, stack):
    global discovered, cur_components
    discovered = []
    components = []
    count = 0
    for key in stack:
        if key not in discovered:
            count+=1
            if len(cur_components) > 0:
                components.append(cur_components)
                cur_components = []
            second_recursion(graph, key)
    if len(cur_components) > 0:
        if cur_components not in components:
            components.append(cur_components)

    return components

def second_recursion(graph, cur_node):
    global discovered, cur_components

    discovered.append(cur_node)
    cur_components.append(cur_node)

    for node in graph[cur_node]:
        if node not in discovered:
            second_recursion(graph, node)


def graph_transpose(graph):
    transpose = {}
    for key in graph: transpose[key] = {}
    
    for node in graph:
        for key in graph[node]:
            transpose[key][node] = graph[node][key]
            
    return transpose

def kosaraju(graph):
    stack = scc_dfs(graph)
    transpose = graph_transpose(graph)
    components = scc_second(transpose, stack)
    return components

len(kosaraju(graph))
# print(kosaraju(graph))

53

In [103]:
count = 0
for node in graph:
    count+=len(graph[node])
        
transpose = graph_transpose(graph)

count2=0
for node in transpose:
    count2+=len(transpose[node])

print(count, count2)
    

4308 4308


In [15]:
# testing code 
# print(18871 + 15753)

s = 'Benjamin H. Kline_d'

print(s[len(s)-1:])

# file_path = 'imdb_network.csv'
# data = pd.read_csv(file_path)
# data = data.values.tolist()

# print(len(data))

# for i in data:
#     print(i)

stack = [1, 2, 3, 4, 5]

stack.pop(0)

print(stack)



[2, 3, 4, 5]
